In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from sklearn.model_selection import train_test_split
import numpy as np
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.stem.wordnet import WordNetLemmatizer
# lemma=WordNetLemmatizer()
# stop=set(stopwords.words('english'))

import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/stacksample.zip", 'r')

zip_ref.extractall()
zip_ref.close()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ques=pd.read_csv('Questions.csv',encoding='iso-8859-1')
ques.head()


,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [ ]:
ques.isnull().sum().sort_values(ascending=False)

ClosedDate      1208257
OwnerUserId       14454
Body                  0
Title                 0
Score                 0
CreationDate          0
Id                    0
dtype: int64

In [ ]:
ques.drop(['OwnerUserId','CreationDate','ClosedDate','Score'],axis=1,inplace=True)
ques.head()

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [ ]:
import re
def ref_html(body):
  regex=re.compile('<.*?>')
  return re.sub(regex,' ',body)

ques['Body']=ques['Body'].apply(ref_html)
ques.head()

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in ...
1,90,Good branching and merging tutorials for Torto...,Are there any really good tutorials explainin...
2,120,ASP.NET Site Maps,Has anyone got experience creating SQL-based...
3,180,Function for creating color wheels,This is something I've pseudo-solved many tim...
4,260,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a...


In [ ]:
df_tags=pd.read_csv('Tags.csv',encoding='iso-8859-1')
df_tags.head()


,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [ ]:
df_tags['Tag'].value_counts().sort_values(ascending=False).head(10)

javascript    124155
java          115212
c#            101186
php            98808
android        90659
jquery         78542
python         64601
html           58976
c++            47591
ios            47009
Name: Tag, dtype: int64

In [ ]:
top10=['javascript','java','c#','php','android','jquery','python','html','c++','ios']


In [ ]:
tag_top10=df_tags[df_tags.Tag.isin(top10)]
print(tag_top10.shape)
tag_top10.head(10)

(826739, 2)


,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#
41,1040,c#
61,1760,c#
74,2250,c#
79,2530,html
97,2780,c#


In [ ]:
tag_top10['Tags']=tag_top10['Tag'].apply(lambda x:x.split())


In [ ]:
tag_top10.drop(columns=['Tag'],inplace=True,axis=1)
tag_top10.head()

,Id,Tags
14,260,[c#]
18,330,[c++]
28,650,[c#]
35,930,[c#]
39,1010,[c#]


In [ ]:
total=pd.merge(ques, tag_top10, on='Id')
print(total.shape)
total.head()

(826739, 4)


,Id,Title,Body,Tags
0,260,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a...,[c#]
1,330,Should I use nested classes in this case?,I am working on a collection of classes used ...,[c++]
2,650,Automatically update version number,I would like the version property of my appli...,[c#]
3,930,How do I connect to a database and loop over a...,What's the simplest way to connect and query ...,[c#]
4,1010,"How to get the value of built, encoded ViewState?",I need to grab the base64-encoded representat...,[c#]


In [ ]:
total['Text'] = total['Title'] + ' ' + total['Body']


In [ ]:
print(total.isnull().sum())

# total[total.astype(str).duplicated('Text')]
print(total.shape)
total.drop_duplicates(subset ="Text",keep = False, inplace = True) 
print(total.shape)
total.reset_index(inplace = True, drop = True) 



Id       0
Title    0
Body     0
Tags     0
Text     0
dtype: int64
(826739, 5)
(604849, 5)


In [ ]:
total.drop(columns=['Id','Title','Body'],inplace=True,axis=1)

In [ ]:
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation

token=ToktokTokenizer()
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
lemma=WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
punctuation
punct='!"$%&\'()*,-./:;<=>?@[\\]^_`{|}~'


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text



In [ ]:
def remove_punc(text):
  punct=set('!"$%&\'()*,-./:;<=>?@[\\]^_`{|}~')
  words=token.tokenize(text)
  filtered = [w for w in words if not w in punct]
  return ' '.join(map(str, filtered))



In [ ]:
def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))

def stopWordsRemove(text):
    stop_words = set(stopwords.words("english"))
    words=token.tokenize(text)
    filtered = [w for w in words if not w in stop_words]
    return ' '.join(map(str, filtered))



In [ ]:

total['Text'] = total['Text'].apply(lambda x: clean_text(x)) 

total['Text'] = total['Text'].apply(lambda x: remove_punc(x)) 

total['Text'] = total['Text'].apply(lambda x: lemitizeWords(x)) 
total['Text'] = total['Text'].apply(lambda x: stopWordsRemove(x)) 



In [ ]:
from keras.preprocessing.text import Tokenizer
text=total['Text'].tolist()
text
token=Tokenizer()
token.fit_on_texts(text)
vocab_size=len(token.word_index)+1
print(vocab_size)

noofwords= total['Text'].apply(lambda x: len(x.split()))
maxlen=max(noofwords)
maxlen

Using TensorFlow backend.


1684129


10489

In [ ]:
# import nltk
# nltk.download('punkt')
# from nltk.tokenize import word_tokenize
# sent_lens_b=[]
# for sent in total['Text']:
#     sent_lens_b.append(len(word_tokenize(sent)))
# max(sent_lens_b)
# X=total['Text']
# print(np.quantile(sent_lens_b,0.90))--335 so selected 350


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


15100

In [ ]:
from keras.preprocessing.sequence import pad_sequences
vocab_size=5000

tokenizer = Tokenizer(num_words= vocab_size)
tokenizer.fit_on_texts(total['Text'])

sequences = tokenizer.texts_to_sequences(total['Text'])
data = pad_sequences(sequences, maxlen)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
y=total['Tags']
multilabel_binarizer = MultiLabelBinarizer()
y_bin = multilabel_binarizer.fit_transform(y)
y_bin.shape

(604849, 10)

In [ ]:
print(data.shape,y_bin.shape)
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout


(604849, 350) (604849, 10)


In [ ]:

model = Sequential()
# Configuring the parameters
model.add(Embedding(vocab_size, output_dim=10, input_length=350))
# model.add(LSTM(128, return_sequences=True))  
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(20))
model.add(Dropout(0.5))
# Adding a dense output layer with  activation
model.add(Dense(10,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['categorical_accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 350, 10)           50000     
_________________________________________________________________
dropout (Dropout)            (None, 350, 10)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 20)                2480      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                210       
Total params: 52,690
Trainable params: 52,690
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
X_final=np.array(data)
y_final=np.array(y_bin)
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((405248, 350), (405248, 10), (199601, 350), (199601, 10))

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=1,batch_size=64)


6332/6332 [==============================] - 819s 129ms/step - loss: 1.5456 - categorical_accuracy: 0.3993 - val_loss: 1.2650 - val_categorical_accuracy: 0.5362


In [ ]:
# y_pred=model.predict_classes(X_test)

# ypre=np.argmax(model.predict(X_test), axis=-1)
# rounded_labels=np.argmax(y_test, axis=1)
rounded_labels


array([8, 5, 5, ..., 5, 8, 6])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(rounded_labels,ypre)


# ypre

0.5361796784585248

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(rounded_labels,ypre)



array([[ 7017, 13697,     2,     0,    32,  1091,   179,     4,   804,
           87],
       [   30, 30035,    22,     0,     5,   270,   457,     0,   451,
          176],
       [    5, 11792,  1819,     0,     2,   107,   722,     0,   210,
          225],
       [    4,  5692,   114,    25,     9,    78,  1478,     1,   253,
           81],
       [    9,  1698,     1,     0,  3155,    47,   238,     1,    16,
         9415],
       [   37,  4395,    13,     0,     2, 23941,   125,     0,  2816,
           62],
       [   23,  4732,   217,     0,    19,   128, 15155,     2,   236,
          234],
       [    4,  1053,    58,     3,     5,    47,  7811,     2,   140,
           66],
       [    9,  4126,    76,     3,     2, 12989,   522,     0,  8431,
           80],
       [    2,  2256,     5,     0,   230,   150,   355,     1,    40,
        17442]])

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(rounded_labels,ypre,average='micro'))
print(f1_score(rounded_labels,ypre,average='macro'))



0.5361796784585248
0.40394565568469387
